# Data Analysis for Lab3 Dataset

## Introduction
In this section, we analyze the yield data for the month of March.
We visualize the actual and predicted values to observe any discrepancies.

## Plotting Yield Data

In [45]:
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import itertools

# Factorial design and screening model 
The following function is the first part of the DoE, making it simple to run the experiment several times to get the right values

In [46]:
def computing(inputs_df):
    # Compute the average and span for each variable in the inputs DataFrame
    # The average is calculated as the midpoint between 'low' and 'high' values
    inputs_df['average'] = inputs_df.apply(lambda z: (z['high'] + z['low']) / 2, axis=1)

    # The span is calculated as half the difference between 'high' and 'low' values
    inputs_df['span'] = inputs_df.apply(lambda z: (z['high'] - z['low']) / 2, axis=1)

    # Encode the data using standardized values for 'low', 'center', and 'high'
    # The encoding formula centers the data around the average and scales it by the span
    inputs_df['encoded_low'] = inputs_df.apply(lambda z: (z['low'] - z['average']) / z['span'], axis=1)
    inputs_df['encoded_center'] = inputs_df.apply(lambda z: (z['center'] - z['average']) / z['span'], axis=1)
    inputs_df['encoded_high'] = inputs_df.apply(lambda z: (z['high'] - z['average']) / z['span'], axis=1)

    # Drop the 'average' and 'span' columns as they are no longer needed for further analysis
    inputs_df = inputs_df.drop(['average', 'span'], axis=1)

    # Display the modified inputs DataFrame
    inputs_df

    # Generate all combinations of -1 and 1 for two variables using itertools.product
    encoded_inputs = list(itertools.product([-1, 1], [-1, 1]))
    
    # Append the tuple (0, 0) five times to the list of encoded inputs
    for i in range(0, 5):
        encoded_inputs.append((0, 0))

    # Create a DataFrame from the list of encoded inputs
    results = pd.DataFrame(encoded_inputs)

    # Reverse the order of the columns in the DataFrame
    results = results[results.columns[::-1]]

    #   Rename the columns to 't' for the first column and 'T' for the second column
    results.columns = ['c', 'T']
    
    # Create a copy of results for real_experiment
    real_experiment = results.copy()
    var_labels = []

    # Loop through the existing variables in inputs_df
    for var in inputs_df.index:
        # Get the label for the variable
        var_label = inputs_df.loc[var]['label']
        var_labels.append(var_label)
        
        # Apply the function to create a new column based on conditions
        real_experiment[var_label] = results.apply(
            lambda z: inputs_df.loc[var]['low'] if z[var] < 0 else 
                    (inputs_df.loc[var]['high'] if z[var] > 0 else 
                        inputs_df.loc[var]['center']),
            axis=1
        )
    
    return real_experiment, results

Define input variables and start guess, as well as reaction rate (yield)

In [47]:
# Define a dictionary to map input variable names to their descriptive labels
inputs_labels = {
    'c': 'concentration',
    'T': 'Temperature'
}

# Define the initial values for concentration, temperature, and reaction rate
c = 2.0  # mM (concentration)
T = 25.0  # °C (temperature)
rate = 5.77  # mole/s (reaction rate)

## Value intervals

Used for the analysis

In [48]:
# Create a list of tuples with each variable's low, center, and high values
dat = [
    ('c', 0.80 * c, c, 1.20 * c),  # Concentration: low is 80% of c, center is c, high is 120% of c
    ('T', 0.80 * T, T, 1.20 * T)    # Temperature: low is 80% of T, center is T, high is 120% of T
]

# Create a Pandas DataFrame from the data list, specifying the column names
inputs_df = pd.DataFrame(dat, columns=['index', 'low', 'center', 'high'])

# Set the 'index' column as the DataFrame index for easier access
inputs_df = inputs_df.set_index('index')

# Map the variable labels to the DataFrame index, providing a default empty string for unmapped values
inputs_df['label'] = inputs_df.index.map(lambda z: inputs_labels.get(z, ''))

# Print the resulting DataFrame to display its contents
print(inputs_df)

real_experiment, results = computing(inputs_df) #Run the function defined earlier

c_array  = real_experiment['concentration']
T_array = real_experiment['Temperature']

        low  center  high          label
index                                   
c       1.6     2.0   2.4  concentration
T      20.0    25.0  30.0    Temperature


**FIRST EXPERIMENT**

In [49]:
y=[4.093000401645197,
   4.77827155867177,
   6.403139782334854,
   8.199982171532273,
   5.847007863003852,
   5.695005284140945,
   5.584338442337459,
   5.806999272226782,
   5.814771183739139] # Data from the first experiment

results['y']= y
results

,c,T,y
0,-1,-1,4.093000
1,1,-1,4.778272
2,-1,1,6.403140
3,1,1,8.199982
4,0,0,5.847008
5,0,0,5.695005
6,0,0,5.584338
7,0,0,5.806999
8,0,0,5.814771


In [50]:
# Data , 4 corners and 5 center points:

df = pd.DataFrame(results,columns=['c','T','y'])

df # Print dataframe

,c,T,y
0,-1,-1,4.093000
1,1,-1,4.778272
2,-1,1,6.403140
3,1,1,8.199982
4,0,0,5.847008
5,0,0,5.695005
6,0,0,5.584338
7,0,0,5.806999
8,0,0,5.814771


Run regression on the data in linear format to get the coeffcient

In [51]:
res1 = smf.ols(formula='y ~ c + T', data=results).fit()
print(res1.summary())
res1 #lägg till step size på ols

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.962
Model:                            OLS   Adj. R-squared:                  0.949
Method:                 Least Squares   F-statistic:                     75.48
Date:                Fri, 11 Oct 2024   Prob (F-statistic):           5.59e-05
Time:                        12:45:21   Log-Likelihood:                 1.3812
No. Observations:                   9   AIC:                             3.238
Df Residuals:                       6   BIC:                             3.829
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.8025      0.085     68.482      0.0

/home/oscar/miniconda3/envs/dhenv/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=9 observations were given.
  return hypotest_fun_in(*args, **kwds)


The following linear approximation is obtained from the regression:  
$\bar{y} = 5.8025 + 0.6205 \cdot c + 1.4330 \cdot T$

To move toward a higher yield, closer to the maximum of the response surface, the response \(y\) can be predicted for any concentration \(c\) and temperature \(T\) based on this flat surface model, where \(y\) is a function of the coded variables \(x_1\) and \(x_2\). The coefficients represent the gradient of the function. By moving 0.6205 units in the direction of \(c\) and 1.4330 units in the direction of \(T\), the path of steepest ascent is followed, leading toward the maximum response. Starting at 0 in coded units, a series of single experiments can be conducted along this path, increasing \(c\) by 1 unit at each step, until the response starts to decrease.

Therefor the following becomes the step size used: \
$\Delta$ = 1.4339/0.6205 = 2.31

In [52]:
Origin = [c,T]
delta= [1.0, 2.31]

march=[]
for i in range(0,len(y)):
    march.append((Origin[0]+(i+1)*delta[0],Origin[1]+(i+1)*delta[1]))

March=pd.DataFrame(march,columns=['c', 'T'])
ypred=res1.predict(March)

March['ypred']=ypred

March

,c,T,ypred
0,3.0,27.31,46.798293
1,4.0,29.62,50.728965
2,5.0,31.93,54.659636
3,6.0,34.24,58.590308
4,7.0,36.55,62.520980
5,8.0,38.86,66.451652
6,9.0,41.17,70.382323
7,10.0,43.48,74.312995
8,11.0,45.79,78.243667


The data indicates that the yield increases with higher temperatures and concentrations, suggesting a need to move in that direction. The coefficients were recalculated multiple times during this process to achieve optimal values. The following table summarizes the attempts:

| Attempt | c   | T          |
|---------|-----|------------|
| 1       | 6.00| 34.24      |
| 2       | 5.00| 25.06      |
| 3       | 4.00| 29.62      |
| 4       | 3.00| 27.31      |
| 5       | 3.00| 30.00      |
| 6       | 3.50| 35.00      |
| 7       | 3.70| 38.00      |
| 8       | 3.75| 40.00      |
| 9       | 3.70| 39.00      |

In the first attempt, the step size was too large, resulting in poor data. As a result, several adjustments were made, stepping back and gradually increasing the concentration until an optimal range was found. Finally, the temperature was incrementally increased for fine-tuning. The last attempt illustrates this approach.

In [53]:
c = 3.70
T  = 39.0

# Create a list of tuples with each variable's low, center, and high values
dat = [
    ('c', 0.8 * c, c, 1.20 * c),  # Concentration: low is 80% of c, center is c, high is 120% of c
    ('T', 0.8 * T, T, 1.20 * T)    # Temperature: low is 80% of T, center is T, high is 120% of T
]

# Create a Pandas DataFrame from the data list, specifying the column names
inputs_df = pd.DataFrame(dat, columns=['index', 'low', 'center', 'high'])

# Set the 'index' column as the DataFrame index for easier access
inputs_df = inputs_df.set_index('index')

# Map the variable labels to the DataFrame index, providing a default empty string for unmapped values
inputs_df['label'] = inputs_df.index.map(lambda z: inputs_labels.get(z, ''))

# Print the resulting DataFrame to display its contents
print(inputs_df)

real_experiment, results = computing(inputs_df)

c_array  = real_experiment['concentration'].tolist()
T_array = real_experiment['Temperature'].tolist()

print(f'c = {c_array}')
print(f'T = {T_array}')

         low  center   high          label
index                                     
c       2.96     3.7   4.44  concentration
T      31.20    39.0  46.80    Temperature
c = [2.9600000000000004, 4.44, 2.9600000000000004, 4.44, 3.7, 3.7, 3.7, 3.7, 3.7]
T = [31.200000000000003, 31.200000000000003, 46.8, 46.8, 39.0, 39.0, 39.0, 39.0, 39.0]


In [54]:
y = [10.198819978308894,
10.524033796374926,
11.039326780371198,
10.87110871563581,
13.91363162951745,
13.92923511074255,
13.748190855517132,
13.802006153111138,
13.806009629601999] # Data from the last experiment

results['y']= y

res1 = smf.ols(formula='y ~ c + T', data=results).fit()
print(res1.summary())
res1

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                 -0.312
Method:                 Least Squares   F-statistic:                   0.04767
Date:                Fri, 11 Oct 2024   Prob (F-statistic):              0.954
Time:                        12:45:21   Log-Likelihood:                -16.909
No. Observations:                   9   AIC:                             39.82
Df Residuals:                       6   BIC:                             40.41
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.4258      0.647     19.217      0.0

/home/oscar/miniconda3/envs/dhenv/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=9 observations were given.
  return hypotest_fun_in(*args, **kwds)


The regression provides the following function: \
$\bar{y} = 12.4258 + 0.0392 \cdot c + 0.2969 \cdot T$ \

The coefficients are noticeably lower compared to the initial attempts, suggesting that the process is now approaching the maximum yield. This indicates a successful optimization of the conditions.

In [55]:
c = 3.70
T  = 39.0
Origin = [c,T]
delta= [1.0, 0.0392/0.2969]

march=[]
for i in range(0,len(y)):
    march.append((Origin[0]+(i+1)*delta[0],Origin[1]+(i+1)*delta[1]))

March=pd.DataFrame(march,columns=['c', 'T'])
ypred=res1.predict(March)

March['ypred']=ypred

March

,c,T,ypred
0,4.7,39.132031,24.228409
1,5.7,39.264062,24.306858
2,6.7,39.396093,24.385306
3,7.7,39.528124,24.463754
4,8.7,39.660155,24.542203
5,9.7,39.792186,24.620651
6,10.7,39.924217,24.699099
7,11.7,40.056248,24.777548
8,12.7,40.188279,24.855996


# Optimization with Latin Hypercube Design (LHD)

Nine new randomized data points are generated using Latin Hypercube Sampling (LHS), with the values randomized within the range defined by the specified low and high values for `c` (concentration) and `T` (temperature).

In [56]:
from doepy import read_write, build

c = March['c'].tolist()
T = March['T'].tolist()
ypred = March['ypred'].tolist()

optmod=build.space_filling_lhs(
{'c':[3.6, 3.8],
'T':[37, 39]},
num_samples = 9
)

optmod
#lhs_df = pd.DataFrame(lhs_design, columns['Factor1', 'Factor2', 'Factor3'])

,c,T
0,3.600,37.50
1,3.625,38.75
2,3.750,38.25
3,3.650,38.50
4,3.725,37.25
5,3.775,39.00
6,3.700,38.00
7,3.800,37.75
8,3.675,37.00


In [57]:
# c_lhs = optmod['c'].tolist()
# T_lhs = optmod['T'].tolist()
c_lhs = [3.725, 3.750, 3.775, 3.700, 3.625, 3.650, 3.800, 3.600, 3.675] # Set the new concentrations and tempatures so it doesnt get over run
T_lhs = [37.25, 39.0, 37.75, 38.25, 37.0, 38.0, 38.75, 37.5, 38.5]
c = c_lhs
T = T_lhs

new_row = pd.DataFrame({'c': [3.7], 'T': [38]})
results_lhs = {'c': c,'T': T}
results_lhs = pd.DataFrame(results_lhs)
results_lhs = pd.concat([results_lhs, new_row], ignore_index=True)
results_lhs

,c,T
0,3.725,37.25
1,3.750,39.00
2,3.775,37.75
3,3.700,38.25
4,3.625,37.00
5,3.650,38.00
6,3.800,38.75
7,3.600,37.50
8,3.675,38.50
9,3.700,38.00


In [58]:
y_opt=[13.66205178878079,
14.241054027404259,
13.730652628229132,
13.871114822697608,
13.734226231398884,
13.965993857986756,
14.152461402645864,
13.9767553245175,
14.002133837397391,
13.833584118310686]

results_lhs['y']=y_opt
results_lhs

,c,T,y
0,3.725,37.25,13.662052
1,3.750,39.00,14.241054
2,3.775,37.75,13.730653
3,3.700,38.25,13.871115
4,3.625,37.00,13.734226
5,3.650,38.00,13.965994
6,3.800,38.75,14.152461
7,3.600,37.50,13.976755
8,3.675,38.50,14.002134
9,3.700,38.00,13.833584


In [59]:
res2_lhs = smf.ols(formula='y ~ c + T + c:T + I(c**2) + I(T**2)', data=results_lhs).fit()
print(res2_lhs.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.929
Method:                 Least Squares   F-statistic:                     24.60
Date:                Fri, 11 Oct 2024   Prob (F-statistic):            0.00420
Time:                        12:45:21   Log-Likelihood:                 20.376
No. Observations:                  10   AIC:                            -28.75
Df Residuals:                       4   BIC:                            -26.94
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    358.2744     80.415      4.455      0.0

/home/oscar/miniconda3/envs/dhenv/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=10 observations were given.
  return hypotest_fun_in(*args, **kwds)


In [60]:
from scipy.optimize import minimize

# Define the function to calculate the y-value
def y_function(params):
    # 'c' is the concentration, 'T' is the temperature, both come from params
    c, T = params
    return (-1) * (res2_lhs.params['Intercept'] +  # Linear model's intercept
                   res2_lhs.params['c'] * c +  # Contribution from concentration (c)
                   res2_lhs.params['T'] * T +  # Contribution from temperature (T)
                   res2_lhs.params['c:T'] * c * T +  # Interaction between concentration and temperature
                   res2_lhs.params['I(c ** 2)'] * c**2 +  # Contribution from concentration squared
                   res2_lhs.params['I(T ** 2)'] * T**2)  # Contribution from temperature squared

# Initial guess for concentration and temperature (adjusted to broaden the search)
initial_guess = [np.mean(c), np.mean(T)]  # Use the average values of concentration and temperature

# Optimize to find the maximum point
bounds = ((min(c), max(c)), (min(T), max(T)))  # Set constraints for concentration and temperature ranges

# Perform the minimization (note: minimizing the negative function to find the maximum)
result = minimize(y_function, initial_guess, bounds=bounds)

# Check if optimization was successful
if result.success:
    max_point = result.x  # Get the values for c and T at the maximum point
    max_value = -result.fun  # Negate to get the maximum value (since we minimized the negative)
    print(f"Maxima: concentration = {max_point[0]:.4f}, Temperature = {max_point[1]:.2f}, Max value = {max_value:.4f}")
else:
    print("No maximum point found.")


Maxima: concentration = 3.6000, Temperature = 39.00, Max value = 14.5718


In [61]:
# Here, the final values obtained will be inserted to get the best possible approximation of the maximum.
# A linear regression model will be created for these data points, based on a polynomial.

optmod = build.space_filling_lhs(
    {'c': [3.6, 3.8],  # Specify the range for concentration 'c' (between 3.6 and 3.8)
    'T': [37, 39]},    # Specify the range for temperature 'T' (between 37 and 39)
    num_samples = 8    # Number of random samples to generate within this range
)

# Convert the resulting sample points into a pandas DataFrame for easier manipulation and analysis
optmod = pd.DataFrame(optmod)
optmod  # Display the generated DataFrame

,c,T
0,3.742857,38.142857
1,3.600000,37.857143
2,3.685714,37.571429
3,3.628571,38.714286
4,3.714286,37.000000
5,3.771429,38.428571
6,3.800000,37.285714
7,3.657143,39.000000


In [62]:
# Create a DataFrame with the new experimental points and concatenate it with the previous "exp_par" DataFrame.

# The commented-out lines convert the columns 'c' and 'T' from optmod into lists
# c_lhs2 = optmod['c'].tolist()
# T_lhs2 = optmod['T'].tolist()

# Manually defining the lists for concentration (c_lhs2) and temperature (T_lhs2)
c_lhs2 = [3.742857142857143, 3.6285714285714286, 3.657142857142857, 
          3.6857142857142855, 3.7714285714285714, 3.6, 3.7142857142857144, 3.8]

T_lhs2 = [37.857142857142854, 37.0, 38.42857142857143, 38.142857142857146, 
          37.285714285714285, 37.57142857142857, 39.0, 38.714285714285715]

# Create a dictionary to store the new experimental points for 'c' and 'T'
results_lhs2 = {'c': c_lhs2, 'T': T_lhs2}

# Convert the dictionary to a pandas DataFrame for easy manipulation and analysis
results_lhs2 = pd.DataFrame(results_lhs2)

# The list 'y_opt1' contains the corresponding output (y-values) for each of the 8 new data points
y_opt1 = [13.820280504274544, 13.603840691714115, 13.783238339994858, 
          13.841772379782215, 13.716442504317829, 14.114163621553518, 
          13.99930132597303, 13.866021170060046]

# Add the 'y' values to the DataFrame as a new column
results_lhs2['y'] = y_opt1

# Display the DataFrame containing only the 8 new data points and their corresponding results
results_lhs2

,c,T,y
0,3.742857,37.857143,13.820281
1,3.628571,37.000000,13.603841
2,3.657143,38.428571,13.783238
3,3.685714,38.142857,13.841772
4,3.771429,37.285714,13.716443
5,3.600000,37.571429,14.114164
6,3.714286,39.000000,13.999301
7,3.800000,38.714286,13.866021


In [63]:
# Concatenate the new results DataFrame (results_lhs2) with the previous DataFrame (results_lhs)
# This combines all the data points (previous and new) into a single DataFrame.
results_lhs_combined = pd.concat([results_lhs, results_lhs2], ignore_index=True)

# Display all 18 points and their corresponding results
results_lhs_combined

,c,T,y
0,3.725000,37.250000,13.662052
1,3.750000,39.000000,14.241054
2,3.775000,37.750000,13.730653
3,3.700000,38.250000,13.871115
4,3.625000,37.000000,13.734226
5,3.650000,38.000000,13.965994
6,3.800000,38.750000,14.152461
7,3.600000,37.500000,13.976755
8,3.675000,38.500000,14.002134
9,3.700000,38.000000,13.833584


In [64]:
res2_lhs_combined = smf.ols(formula='y ~ c + T + c:T + I(c**2) + I(T**2)', data=results_lhs_combined).fit() # data=March).fit()
res2_lhs_combined.summary()

/home/oscar/miniconda3/envs/dhenv/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=18 observations were given.
  return hypotest_fun_in(*args, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.659
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                     4.636
Date:                Fri, 11 Oct 2024   Prob (F-statistic):             0.0138
Time:                        12:49:06   Log-Likelihood:                 16.108
No. Observations:                  18   AIC:                            -20.22
Df Residuals:                      12   BIC:                            -14.87
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    240.0428    144.983      1.656      0.124     -75.847     555.933
c           -127.7831     59.568     -2.145      0.053    -257.570       2.003
T              0.3881      5.908      0.066      0.949     -12.484      13.260
c:T           -0.1470      1.333     -0.110      0.914      -3.050       2.756
I(c ** 2)     17.8923     11.458      1.562      0.144      -7.071      42.856
I(T ** 2)      0.0052      0.100      0.052      0.959      -0.213       0.224
==============================================================================
Omnibus:                        1.686   Durbin-Watson:                   1.351
Prob(Omnibus):                  0.430   Jarque-Bera (JB):                0.707
Skew:                          -0.479   Prob(JB):                        0.702
Kurtosis:                       3.152   Cond. No.                     7.67e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.67e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
# Define the function to calculate the y-value
def y_function(params, res2):
    # 'c' is the concentration, 'T' is the temperature, both come from params
    c, T = params
    return (-1) * (res2['Intercept'] +  # Linear model's intercept
                   res2_lhs_combined.params['c'] * c +  # Contribution from concentration (c)
                   res2_lhs_combined.params['T'] * T +  # Contribution from temperature (T)
                   res2_lhs_combined.params['c:T'] * c * T +  # Interaction between concentration and temperature
                   res2_lhs_combined.params['I(c ** 2)'] * c**2 +  # Contribution from concentration squared
                   res2_lhs_combined.params['I(T ** 2)'] * T**2)  # Contribution from temperature squared

# Initial guess for concentration and temperature (adjusted to broaden the search)
initial_guess = [np.mean(c), np.mean(T)]  # Use the average values of concentration and temperature

# Optimize to find the maximum point
bounds = ((min(c), max(c)), (min(T), max(T)))  # Set constraints for concentration and temperature ranges

# Perform the minimization (note: minimizing the negative function to find the maximum)
result = minimize(y_function, initial_guess, bounds=bounds)

# Check if optimization was successful
if result.success:
    max_point = result.x  # Get the values for c and T at the maximum point
    max_value = -result.fun  # Negate to get the maximum value (since we minimized the negative)
    print(f"Maxima: concentration = {max_point[0]:.4f}, Temperature = {max_point[1]:.2f}, Max value = {max_value:.4f}")
else:
    print("No maximum point found.")